# Data Exploration and Wrangling
In this notebook, I explore the data of the columns I chose in the raw data analysis notebook. Here the data is cleaned and processed. Some new columns are created that will be used for the machine learning algorithm using existing columns. For example, the date is split into year, month, and day. The data is then inspected to appropriate the data types for each column.

In [1]:
# import all libraries
import pandas as pd
import numpy as np
from thefuzz import fuzz, process
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer

In [2]:
# import data
collisions = pd.read_csv('accident_data.csv')

C:\Users\mende\AppData\Local\Temp\ipykernel_6676\3500313085.py:2: DtypeWarning: Columns (21,31,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions = pd.read_csv('accident_data.csv')


In [3]:
# drop unnamed column
collisions = collisions.drop('Unnamed: 0', axis = 1) 

## Exploratory Analysis

In [5]:
# view data size
collisions.shape

(4911120, 60)

In [6]:
# inspect data types
collisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911120 entries, 0 to 4911119
Data columns (total 60 columns):
 #   Column                Dtype  
---  ------                -----  
 0   CASE_ID               int64  
 1   AT_FAULT              object 
 2   PARTY_SEX             object 
 3   PARTY_AGE             int64  
 4   PARTY_SOBRIETY        object 
 5   PARTY_DRUG_PHYSICAL   object 
 6   DIR_OF_TRAVEL         object 
 7   PARTY_SAFETY_EQUIP_1  object 
 8   PARTY_SAFETY_EQUIP_2  object 
 9   FINAN_RESPONS         object 
 10  SP_INFO_2             object 
 11  OAF_VIOLATION_CODE    object 
 12  OAF_VIOL_CAT          object 
 13  OAF_1                 object 
 14  OAF_2                 object 
 15  MOVE_PRE_ACC          object 
 16  VEHICLE_YEAR          float64
 17  VEHICLE_MAKE          object 
 18  STWD_VEHICLE_TYPE     object 
 19  CHP_VEH_TYPE_TOWING   float64
 20  CHP_VEH_TYPE_TOWED    object 
 21  RACE                  object 
 22  COLLISION_DATE        float64
 23  COLLISI

In [7]:
# view the count of na's
na_count = collisions.isna().sum()

na_count[na_count>0]

PARTY_DRUG_PHYSICAL       11159
FINAN_RESPONS            294792
OAF_VIOLATION_CODE       312384
VEHICLE_YEAR             497839
VEHICLE_MAKE                 44
CHP_VEH_TYPE_TOWING      212539
CHP_VEH_TYPE_TOWED      4771117
RACE                     915274
COLLISION_DATE             1825
COLLISION_TIME             1825
REPORTING_DISTRICT      3099574
DAY_OF_WEEK                1825
CHP_SHIFT                  1825
POPULATION                 1825
CNTY_CITY_LOC              1825
BEAT_TYPE                  1825
CHP_BEAT_TYPE              1825
DISTANCE                   1825
DIRECTION               1227949
INTERSECTION               1825
WEATHER_1                  1825
WEATHER_2                  1825
STATE_HWY_IND              7313
LOCATION_TYPE           4779506
RAMP_INTERSECTION       4779506
SIDE_OF_HWY             4779540
PARTY_COUNT                1825
PRIMARY_COLL_FACTOR        1825
PCF_CODE_OF_VIOL           1825
PCF_VIOL_CATEGORY          1825
TYPE_OF_COLLISION          1825
MVIW    

Based on missing values `CHP_VEH_TYPE_TOWED`, `REPORTING_DISTRICT`, `LOCATION_TYPE`,`RAMP_INTERSECTION`, `SIDE_OF_HWY`,
`PEDESTRIAN_ACCIDENT`, `BICYCLE_ACCIDENT`, `MOTORCYCLE_ACCIDENT`, `TRUCK_ACCIDENT` will all be removed since most values are missing. `ALCOHOL_INVOLVED` will stay because in general, this is a feature we want to consider in a collision. A lot of rows have 1825 missing values. Given the size of the data frame, dropping the rows associated with 1825 won't significantly impact our final model.

In [8]:
# count dash and blanks
dash_blank_count = ((collisions=='-')| (collisions=='')).sum(axis=0)

print(dash_blank_count[dash_blank_count>0])

PARTY_SEX                785833
PARTY_SOBRIETY           121034
PARTY_DRUG_PHYSICAL     3903904
DIR_OF_TRAVEL            140340
PARTY_SAFETY_EQUIP_1     831528
PARTY_SAFETY_EQUIP_2     881516
SP_INFO_2                645982
OAF_VIOLATION_CODE      4598728
OAF_1                    211676
OAF_2                   4815859
MOVE_PRE_ACC              86713
VEHICLE_MAKE            1182253
STWD_VEHICLE_TYPE        375619
REPORTING_DISTRICT            3
INTERSECTION              34877
WEATHER_1                 18250
WEATHER_2               4792476
RAMP_INTERSECTION         57800
PRIMARY_COLL_FACTOR       16284
PCF_CODE_OF_VIOL        4908887
TYPE_OF_COLLISION         30582
MVIW                      21817
PED_ACTION                 3025
ROAD_SURFACE              28284
ROAD_COND_1               23320
ROAD_COND_2             4879286
LIGHTING                  14247
CONTROL_DEVICE            21974
dtype: int64


Based on dashes and blanks `OAF_VIOLATION_CODE`, `OAF_2`,`WEATHER_2`, `PCF_CODE_OF_VIOL`, `ROAD_COND_2` will be removed due to the high number of missing information. 

In [9]:
# check if any rows are all NA
all_na = collisions.isna().sum(axis=1) == 59 # 59 since all rows have a case id

print('The number of rows with all missing values:', len(all_na[all_na].index.tolist()))

The number of rows with all missing values: 0


## Data Cleaning

In [10]:
# update data to drop columns with excessive missing values
col_to_drop = ['CHP_VEH_TYPE_TOWED', 'REPORTING_DISTRICT', 'LOCATION_TYPE',
               'RAMP_INTERSECTION', 'SIDE_OF_HWY', 'PEDESTRIAN_ACCIDENT',
               'BICYCLE_ACCIDENT','MOTORCYCLE_ACCIDENT','TRUCK_ACCIDENT',
               'OAF_VIOLATION_CODE', 'OAF_2','WEATHER_2', 'PCF_CODE_OF_VIOL',
               'CNTY_CITY_LOC','ROAD_COND_2']

collisions = collisions.drop(col_to_drop, axis = 1)

### Missing values

In [11]:
# fill values for object dtpyes
obj_dtype = collisions.select_dtypes(include='object').columns
collisions[obj_dtype] = collisions[obj_dtype].fillna('Not Stated')


In [12]:
# drop rows with missing values associated with 1825
collisions = collisions.dropna(subset = ['COLLISION_DATE'])


In [13]:
# drop rows with missing latitude and longitude since this information will be important for the tool later in the project
collisions = collisions.dropna(subset = ['LATITUDE'])

In [14]:
# in order to not introduce bias into the data, missing values for VEHICLE_YEAR will not be imputed with the mean. Using KNN or other advanced imputation strategies would not be appropriate since relevant data to impute this information is not available
collisions = collisions.dropna(subset = ['VEHICLE_YEAR'])

In [15]:
# missing values for CHP_VEH_TYPE_TOWING will be replaced with 99 to reflect an unknown value 
collisions['CHP_VEH_TYPE_TOWING'] = collisions['CHP_VEH_TYPE_TOWING'].fillna(99)

In [16]:
# count na's
na_count = collisions.isna().sum()

na_count[na_count>0]

Series([], dtype: int64)

In [17]:
# view the new size of the data
collisions.shape

(3082221, 45)

### Data Manipulation

In [18]:
# fill dashes and blanks with 'Not Stated' since they mean the same in the data dictionary

# define a function for a generic data frame
def replace_dash_blank(df, replacement_value):
        new_df = df.replace(['-','',' ', '- '], replacement_value)
        return new_df

In [19]:
# call the function on collisions data frame
collisions_no_missing = replace_dash_blank(collisions, 'Not Stated')

In [20]:
# replace Not Stated with N in alchohol_involved
collisions_no_missing['ALCOHOL_INVOLVED'] = collisions_no_missing['ALCOHOL_INVOLVED'].replace('Not Stated', 'N')

In [21]:
# look at unique values to find values that have to be changed to fit contextual information

for col in collisions_no_missing:
    unique_vals = collisions_no_missing[col].unique()
    print(col,':','\n', unique_vals)

CASE_ID : 
 [6698645 8008483 8008487 ... 9656181 9656225 9656269]
AT_FAULT : 
 ['Y' 'N']
PARTY_SEX : 
 ['M' 'Not Stated' 'F' 'X']
PARTY_AGE : 
 [ 31  44  27 998  41  33  47  25  22  39  64  68  36  32  77  21  55  48
  38  67  56  34  19  58  50  35  52  40  18  37  23  60  62  49  74  65
  72  46  29  57  53  28  78  66  70  24  20  63  30  43  26  45  76  59
  61  17  83  69  16  86  79  71  51  54  12  84  75  87  80  81  42  15
  85  73  88  93  13  89  82  10  91   0  90  92  14 119  95 101  98   1
   2 120  96 121   8  94  11   5   9   7   4  99  97 122 104 117 100   3
 112 103   6 123 118 102 105 110 109 107 113 124 111 114 125 106 108]
PARTY_SOBRIETY : 
 ['B' 'A' 'H' 'G' 'D' 'C' 'Not Stated']
PARTY_DRUG_PHYSICAL : 
 ['Not Stated' 'H' 'G' 'E' 'I' 'F']
DIR_OF_TRAVEL : 
 ['N' 'W' 'S' 'E' 'Not Stated']
PARTY_SAFETY_EQUIP_1 : 
 ['M' 'Not Stated' 'L' 'P' 'N' 'B' 'G' 'C' 'R' 'K' 'H' 'U' 'V' 'X' 'A' 'E'
 'Y' 'W' 'S' 'D' 'Q' 'F' 'J']
PARTY_SAFETY_EQUIP_2 : 
 ['G' 'Not Stated' 'B' 'H' 'C

##### Notes:
`PARTY_SEX` has a value 'X' that must be changed to 'Not Stated'  
`PARTY_AGE` has a 998 value to represent a missing age. 273364 rows have 998
`VEHICLE_MAKE` has different spellings for the same manufacturer  
`COLLISION_DATE` must be separated into year, day, and month  
`COLLISION_TIME` must be separated to hour and minute  
`CHP_BEAT_TYPE` has mixed data types  
`MVIW` has numbers and letters when only letters should be present based on the data description  
`NOT_PRIVATE_PROPERTY` is all yes so it will be dropped


In [22]:
# change X to Not Stated in PARTY_SEX
collisions_no_missing['PARTY_SEX'] = collisions_no_missing['PARTY_SEX'].replace('X', 'Not Stated')

In [23]:
# split COLLISION_DATE to year, month, day
collisions_no_missing['COLLISION_DATE'] = collisions_no_missing['COLLISION_DATE'].astype('string')

collisions_no_missing['COLLISION_YEAR'] = collisions_no_missing['COLLISION_DATE'].str.slice(0,4)
collisions_no_missing['COLLISION_MONTH'] = collisions_no_missing['COLLISION_DATE'].str.slice(4,6)
collisions_no_missing['COLLISION_DAY'] = collisions_no_missing['COLLISION_DATE'].str.slice(6,8)


In [24]:
# split time to hour and minute
collisions_no_missing['COLLISION_TIME'] = collisions_no_missing['COLLISION_TIME'].astype('string')

collisions_no_missing['COLLISION_HOUR'] = collisions_no_missing['COLLISION_TIME'].str.slice(0,2)
collisions_no_missing['COLLISION_MINUTE'] = collisions_no_missing['COLLISION_TIME'].str.slice(2,4)


In [25]:
# drop columns that are no longer needed
collisions_no_missing = collisions_no_missing.drop(['COLLISION_DATE','NOT_PRIVATE_PROPERTY','COLLISION_TIME'], axis=1)

In [26]:
# replace 0 with '0' in CHP_BEAT_TYPE
collisions_no_missing['CHP_BEAT_TYPE'] = collisions_no_missing['CHP_BEAT_TYPE'].replace(0, '0')

In [27]:
# change numbers in MVIW to letter 0 = A, 1 = B,... per the data description
mviw_dict = {'0': 'A', '1': 'B', '2': 'C', '3': 'D', '4': 'E', '5': 'F', '6': 'G', '7': 'H', '8': 'I', '9': 'J'}

collisions_no_missing['MVIW'] = collisions_no_missing['MVIW'].replace(mviw_dict)

In [28]:
# look at the names of vehicle makes
collisions_no_missing['VEHICLE_MAKE'].unique().tolist()

['NISSAN',
 'AUDI',
 'Not Stated',
 'FORD',
 'SATURN',
 'CHEVROLET',
 'TOYOTA',
 'KIA',
 'OLDSMOBILE',
 'RAM',
 'MAZDA',
 'BMW',
 'HONDA',
 'LEXUS',
 'ACURA',
 'MERCEDES-BENZ',
 ' FORD',
 ' TOYOTA',
 ' CHEVROLET',
 ' DODGE',
 '  HONDA',
 'MERCURY',
 'INFINITI',
 'VOLVO',
 'CADILLAC',
 'ISUZU',
 'GMC',
 ' CHRYSLER',
 'DODGE',
 ' HONDA',
 'SUBARU',
 'MITSUBISHI',
 'HYUNDAI',
 'BUICK',
 'VOLKSWAGEN',
 'CHRYSLER',
 'SUZUKI',
 'JEEP',
 'FIAT',
 'HARLEY DAVIDSON',
 'YAMAHA',
 'JAGUAR',
 'TRIUMPH',
 'SCION',
 'TESLA',
 'PONTIAC',
 'HYPER',
 'FREIGHTLINER',
 'STATE BICYCLE',
 'PETERBUILT',
 'INTERNATIONAL',
 'GROVE',
 'LINCOLN',
 'MINI',
 'KMM KOVATCH',
 'NEWFLYER',
 'PLYMOUTH',
 'KAWASAKI',
 'KENWORTH',
 ' GMC',
 'BIKE',
 'LAND ROVER',
 'SURGE',
 'PORSCHE',
 'CHEV',
 'KIA ',
 'ACUR',
 'VESP',
 'TOYT',
 'DODG',
 'BMW ',
 'HD  ',
 'MAZD',
 'SCIO',
 'MERZ',
 'NISS',
 'LEXS',
 'HOND',
 'SAFARI',
 'VOLK',
 'GMC ',
 'NWFLR',
 'TESL',
 'INFI',
 'HYUN',
 'LNDR',
 'YAMA',
 'SUBA',
 'GIAN',
 'NFLY',
 '

In [29]:
# change incorrect vehicle spelling to correct spelling in vehicle_make column
car_makes = {r'.*for.*': 'Ford', r'.*toy.*': 'Toyota', r'.*sub.*': 'Subaru', r'.*nis.*': 'Nissan', r'.*at.*': 'Saturn',
             r'.*chev.*': 'Chevrolet', r'.*kia.*': 'Kia', r'.*old.*': 'Oldsmobile', r'.*ram.*': 'Dodge', r'.*dod.*': 'Dodge',
             r'.*maz.*': 'Mazda', r'.*hon.*': 'Honda', r'.*lex.*': 'Lexus', r'.*acu.*': 'Acura', r'.*rced.*': 'Mercedes',
             r'.*rcu.*': 'Mercury', r'.*inf.*': 'Infinity', r'.*vol.*': 'Volvo', r'.*cad.*': 'Cadillac', r'.*isu.*': 'Isuzu',
             r'.*chr.*': 'Chrysler',r'.*hyu.*': 'Hyundai',r'.*bui.*': 'Buick',r'.*wag.*': 'Volkswagen',r'.*suz.*': 'Suzuki',
             r'.*gmc.*': 'GMC',r'.*jeep.*': 'Jeep',r'.*fiat.*': 'Fiat',r'.*Har.*': 'Harley Davidson',r'.*yam.*': 'Yamaha',
             r'.*jag.*': 'Jaguar',r'.*tri.*': 'Triumph',r'.*sci.*': 'Scion',r'.*tes.*': 'Tesla',r'.*tsla.*': 'Tesla',
             r'.*pon.*': 'Pontiac',r'.*bic.*': 'Bicycle',r'.*bike.*': 'Bicycle',r'.*frei.*': 'FreightLiner',r'.*peter.*': 'Peterbuilt',
             r'.*int.*': 'International',r'.*lin.*': 'Lincoln',r'.*mini.*': 'Mini',r'.*newf.*': 'Newflyer',r'.*ply.*': 'Plymouth',
             r'.*kaw.*': 'Kawasaki',r'.*ken.*': 'Kenworth',r'.*land.*': 'Land Rover',r'.*por.*': 'Porsche', 'HD': 'Harley Davidson',
             'MERZ': 'Mercedez','VOLK': 'Volkswagen','NWFLR': 'Newflyer','LNDR': 'Land Rover','NEW FLYER': 'Newflyer','SCHWINN': 'Bike',
             'SAAB': 'Saab','TSMR': 'Tesla', r'.*winne.*': 'Winnebago',r'.*hum.*': 'Hummer','STRN': 'Saturn','FRHT': 'FreightLiner',
             r'.*fer.*': 'Ferrari','PTRB': 'Peterbuilt',r'.*range.*': 'Range Rover','NEWFL': 'Newflyer','TRAVERSE':'Chevrolet',
             'thom': 'Thomas Built', 'dat': 'Datsun',r'.*kw.*': 'Kawasaki',r'.*duc.*': 'Ducati','SCHW':'Bike', 'GM': 'GMC', 
            r'.*alf.*': 'Alfa Romeo','NWFRL':'Newflyer', 'auto':'Not Stated', r'.*bm.*': 'BMW', r'.*bent.*': 'Bentley',
             'TYMC': 'Toyota',r'.*grum.*': 'Grumman','TREK': 'Subaru',r'.*mase.*': 'Maserati', r'.*will.*': 'Jeep', 
             'VW': 'Volkswagen','FRGHT':'FreightLiner', 'BB':'Blue Bird'}

collisions_no_missing['VEHICLE_MAKE'] = collisions_no_missing['VEHICLE_MAKE'].astype('string')

for pattern, replacement in car_makes.items():
    collisions_no_missing['VEHICLE_MAKE'] = collisions_no_missing['VEHICLE_MAKE'].str.replace(pattern, replacement, case = False, regex = True)

In [30]:
# check how many vehicle manufacturers there are
print(f"Number of unique manufacturers: \n {len(collisions_no_missing['VEHICLE_MAKE'].unique().tolist())}")

Number of unique manufacturers: 
 4808


There are still too many different spellings for car makes. Replacing with regular expressions will take too long to process this information using thefuzz. After some research, I found a package called Levenshtein that can help group similar strings to a desired string. The following code uses thefuzz to change incorrect car make spelling.

In [31]:
# make a list of correctly spelled car makes
manuf = ['Ford','Toyota','Subaru','Nissan','Saturn','Chevrolet','Kia','Oldsmobile','Dodge','Mazda','Honda','Lexus',
             'Acura','Mercedes','Mercury','Infinity','Volvo','Cadillac','Isuzu','Chrysler','Hyundai','Buick','Volkswagen',
             'Suzuki','GMC','Jeep','Fiat','Harley Davidson','Yamaha','Jaguar','Triumph','Scion','Tesla','Pontiac','Bicycle',
             'FreightLiner','Peterbuilt','International','Lincoln','Mini Cooper','Newflyer','Plymouth','Mercedez','Land Rover',
             'Saab','Winnebago','Hummer','Ferrari','Range Rover','Thomas Built','Datsun','Kawasaki','Ducati','GMC','Alfa Romeo',
             'BMW','Bentley','Grumman','Maserati','Kenworth','Lucid','Rivian','Nikola','Polaris','Can-Am','John Deere','Smart',
             'Geo','KTM','Caterpillar','Unknown Trailer','Victory','Not Stated','Daewoo','Rolls-Royce','Lamborghini', 'Unknown Motorcycle',
             'Bus','Electric','Genesis','McLaren','Audi', 'Mitsubishi', 'Porsche', 'Blue Bird', 'Gillig','El Dorado','Mack',
             'Fleetwood']

# create a function that can be used for any data frame
def replace_misspled_words(df, col, correct_words, scorer, similarity):
    # make a copy of df to not change the original df
    new_df = df.copy()
    # give a matched word and score for every row in the col
    def correct_word(x):
        matched, score = process.extractOne(x, correct_words, scorer = scorer)
        if score > similarity: 
            return matched
        else:
            return x
    # apply correct_word to new_df
    new_df[col] = new_df[col].apply(correct_word)
    return new_df


In [32]:
# replace words with replace_misspled_words functions
collisions_clean_make = replace_misspled_words(collisions_no_missing, 'VEHICLE_MAKE',manuf,fuzz.WRatio, 60)

In [33]:
# view the number of unique car makes
print(f"Unique Car Makes: \n {len(collisions_clean_make['VEHICLE_MAKE'].unique().tolist())}")

# count the number of car makes that are not in the manf list
print(f"Count of car makes not in manufacturer list: \n {(~collisions_clean_make['VEHICLE_MAKE'].isin(manuf)).sum()}")

Unique Car Makes: 
 2245
Count of car makes not in manufacturer list: 
 9193


In [34]:
# view the top 20 makes not delagated a correct name
not_in_manf =  ~collisions_clean_make['VEHICLE_MAKE'].isin(manuf)

print(collisions_clean_make[not_in_manf]['VEHICLE_MAKE'].value_counts().head(50))

VEHICLE_MAKE
MIFU                 760
AUTC                 437
WSTR                 318
STRG                 217
WEST                 169
WANCO                160
MBZ                  135
KUBOTA               107
SPCNS                106
PREVOST              105
SPECIALIZED          100
VANHOOL               90
VESP                  73
VESPA                 67
SPECIALI              63
OTHER - DOMESTIC      61
JAYCO                 60
OTHER - MOTORHOME     58
UDXX                  57
HUSQ                  57
HYTR                  55
FRTL                  53
FHRT                  52
HUSQVARNA             51
PREV                  50
FRGT                  49
WSTS                  46
HUFFY                 46
NEW HOLL              43
ORION                 43
PIRC                  42
OTHER - MOPED         41
MONGOOSE              40
PTBR                  38
SMEAL                 37
OSHK                  37
VNHL                  36
VAN HOOL              34
ICRP                  34
WRKCC       

The rest of the makes are either unknown or not a vehicle so, they will be replaced with 'Unknown' to differentiate with 'Not Stated'. This is also only 9193 of 3082221 instances which is .3% of the data. 

In [35]:
# replace the rest of vehicle makes with "Unknown"
mask = ~collisions_clean_make['VEHICLE_MAKE'].isin(manuf)

collisions_clean_make.loc[mask, 'VEHICLE_MAKE'] = 'Unknown'

In [36]:
# view final list of vehicle makes
collisions_clean_make['VEHICLE_MAKE'].unique().tolist()

['Nissan',
 'Audi',
 'Saturn',
 'Ford',
 'Chevrolet',
 'Toyota',
 'Kia',
 'Oldsmobile',
 'Dodge',
 'Mazda',
 'BMW',
 'Honda',
 'Lexus',
 'Acura',
 'Mercedes',
 'Mercury',
 'Infinity',
 'Volvo',
 'Cadillac',
 'Suzuki',
 'GMC',
 'Chrysler',
 'Subaru',
 'Hyundai',
 'Buick',
 'Jeep',
 'Harley Davidson',
 'Yamaha',
 'Jaguar',
 'Triumph',
 'Scion',
 'Tesla',
 'Pontiac',
 'Mini Cooper',
 'Lincoln',
 'Land Rover',
 'Newflyer',
 'Plymouth',
 'Kawasaki',
 'Kenworth',
 'Bicycle',
 'Geo',
 'Porsche',
 'Unknown',
 'Mercedez',
 'Saab',
 'Rivian',
 'Unknown Trailer',
 'Can-Am',
 'John Deere',
 'Gillig',
 'Winnebago',
 'Smart',
 'Peterbuilt',
 'Mitsubishi',
 'Hummer',
 'FreightLiner',
 'Ferrari',
 'Thomas Built',
 'Not Stated',
 'Victory',
 'Range Rover',
 'Fleetwood',
 'KTM',
 'Volkswagen',
 'Unknown Motorcycle',
 'International',
 'Alfa Romeo',
 'Grumman',
 'Maserati',
 'Bentley',
 'Electric',
 'Bus',
 'Mack',
 'Genesis',
 'McLaren',
 'Daewoo',
 'Polaris',
 'Rolls-Royce',
 'Blue Bird',
 'Ducati',
 '

In [37]:
# reset index to account for all dropped rows
collisions_clean_make = collisions_clean_make.reset_index(drop = True)

In [38]:
# view 5 instances
collisions_clean_make.head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,LIGHTING,CONTROL_DEVICE,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE
0,6698645,Y,M,31,B,Not Stated,N,M,G,N,...,C,A,Y,34.57264,118.04491,2018,09,30,19,45
1,6698645,N,M,44,A,Not Stated,N,M,G,Y,...,C,A,Y,34.57264,118.04491,2018,09,30,19,45
2,8008483,Y,M,27,B,Not Stated,W,M,G,N,...,C,A,Y,34.21760,119.18680,2020,10,05,22,36
3,8008483,N,Not Stated,998,H,H,W,Not Stated,Not Stated,O,...,C,A,Y,34.21760,119.18680,2020,10,05,22,36
4,8008483,N,Not Stated,998,H,H,W,Not Stated,Not Stated,O,...,C,A,Y,34.21760,119.18680,2020,10,05,22,36


In [39]:
# view counts for each age
print(collisions_clean_make['PARTY_AGE'].value_counts())

PARTY_AGE
998    273364
22      79075
24      78653
23      78564
25      78449
        ...  
124         2
111         1
125         1
106         1
108         1
Name: count, Length: 125, dtype: int64


Replacing 998 with the average or median would not be ideal since there is an overwhelming amount of missing ages and that could affect the statistical analysis we will do later. Instead, we will impute these values using Iterative imputer and relevant features such as `VEHICLE_MAKE` and `COLLISION_HOUR`. Since the data set is large, KNN Imputer would take too long to complete the task. However, it's still important to use the least amount of features but highly relevant. If time allows at the end of the project, this is an area we can improve by using correlation to select the most important columns to impute age.


In [40]:
# choose subset of columns for features
columns = ['PARTY_AGE','PARTY_SEX','VEHICLE_MAKE','VEHICLE_YEAR','COLLISION_HOUR','RACE']

age_data = collisions_clean_make[columns]

In [41]:
# inspect features
age_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3082221 entries, 0 to 3082220
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   PARTY_AGE       int64  
 1   PARTY_SEX       object 
 2   VEHICLE_MAKE    object 
 3   VEHICLE_YEAR    float64
 4   COLLISION_HOUR  string 
 5   RACE            object 
dtypes: float64(1), int64(1), object(3), string(1)
memory usage: 141.1+ MB


In [42]:
# change dtypes 
age_data.loc[:, 'COLLISION_HOUR'] = age_data['COLLISION_HOUR'].astype('float64')

In [43]:
# change 998 to NA to impute
age_data.loc[age_data['PARTY_AGE'] == 998, 'PARTY_AGE'] = np.nan

In [44]:
# verify 988 have been replaced with NA
age_data.isna().sum()

PARTY_AGE         273364
PARTY_SEX              0
VEHICLE_MAKE           0
VEHICLE_YEAR           0
COLLISION_HOUR         0
RACE                   0
dtype: int64

In [45]:
# one hot encode categorical data

# select columns to one hot encode
cat_cols = ['PARTY_SEX','VEHICLE_MAKE','RACE']

# instantiate object 
ohe = OneHotEncoder(sparse_output = False)

# fit transform the data
encoded_data = ohe.fit_transform(age_data[cat_cols])

# get columns to make a data frame
columns = ohe.get_feature_names_out()

encoded_df = pd.DataFrame(encoded_data, columns=columns)

# join encoded data frame with original age_data
encoded_age_df = pd.concat([age_data, encoded_df], axis = 1)

In [46]:
# drop columns we dont need anymore
encoded_age_df = encoded_age_df.drop(cat_cols, axis = 1)

In [47]:
# scale age_data numeric cols

num_cols = ['VEHICLE_YEAR', 'COLLISION_HOUR']

# instantiate object
scaler = StandardScaler()

scaled_num_age_data = scaler.fit_transform(encoded_age_df[num_cols])

scaled_num_age_df = pd.DataFrame(scaled_num_age_data, columns=num_cols)

prepared_age_df = pd.concat([encoded_age_df, scaled_num_age_df], axis = 1)

In [48]:
# drop columns not needed
prepared_age_df = prepared_age_df.drop(num_cols, axis = 1)

The size of the data frame will most likely impact the time it takes to impute the missing values for `PARTY_AGE`. Therefore, we will reduce the number of columns using PCA and only use 95% of the variance explained by the features.



In [49]:
# reduce dimensionality

age_features = prepared_age_df.drop('PARTY_AGE', axis = 1)

# instanciate object of the PCA class
pca = PCA(n_components=.95)

# fit PCA
reduced_age_features = pca.fit_transform(age_features)


In [50]:
# make df of reduced features
reduced_age_features_df = pd.DataFrame(reduced_age_features)

In [51]:
# join Age column with new features
prepared_age_df = pd.concat([prepared_age_df['PARTY_AGE'],reduced_age_features_df], axis = 1)

In [52]:
# impute missing values with IterativeImputer

# Convert column names to strings
prepared_age_df.columns = prepared_age_df.columns.astype(str)

# instanciate imputer object
imputer = IterativeImputer(estimator=LinearRegression())

# fit and transform the data
imputed_age_data = imputer.fit_transform(prepared_age_df)

# create new imputed df
imputed_age = pd.DataFrame(imputed_age_data, columns = prepared_age_df.columns)


In [53]:
# view missing values
imputed_age.isna().sum()

PARTY_AGE    0
0            0
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0
10           0
11           0
12           0
13           0
14           0
15           0
16           0
17           0
18           0
19           0
20           0
21           0
22           0
23           0
24           0
25           0
26           0
dtype: int64

In [54]:
#replace age column with missing values to imputed age column
collisions_clean_make['PARTY_AGE'] = imputed_age['PARTY_AGE']

#### Appropriating Data Types

In [55]:
# view data types
collisions_clean_make.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3082221 entries, 0 to 3082220
Data columns (total 47 columns):
 #   Column                Dtype  
---  ------                -----  
 0   CASE_ID               int64  
 1   AT_FAULT              object 
 2   PARTY_SEX             object 
 3   PARTY_AGE             float64
 4   PARTY_SOBRIETY        object 
 5   PARTY_DRUG_PHYSICAL   object 
 6   DIR_OF_TRAVEL         object 
 7   PARTY_SAFETY_EQUIP_1  object 
 8   PARTY_SAFETY_EQUIP_2  object 
 9   FINAN_RESPONS         object 
 10  SP_INFO_2             object 
 11  OAF_VIOL_CAT          object 
 12  OAF_1                 object 
 13  MOVE_PRE_ACC          object 
 14  VEHICLE_YEAR          float64
 15  VEHICLE_MAKE          object 
 16  STWD_VEHICLE_TYPE     object 
 17  CHP_VEH_TYPE_TOWING   float64
 18  RACE                  object 
 19  DAY_OF_WEEK           float64
 20  CHP_SHIFT             float64
 21  POPULATION            float64
 22  BEAT_TYPE             float64
 23  CHP_BEA

In [56]:
# round age values to nearest int
collisions_clean_make['PARTY_AGE'] = collisions_clean_make['PARTY_AGE'].round().astype('int64')

In [57]:
# make numeric columns to objects
to_obj = ['DAY_OF_WEEK','BEAT_TYPE','CHP_BEAT_TYPE','PCF_VIOL_CATEGORY']

collisions_clean_make[to_obj] = collisions_clean_make[to_obj].astype('object')

In [68]:
# change to int
str_cols = ['COLLISION_YEAR','COLLISION_MONTH','COLLISION_DAY','COLLISION_HOUR','COLLISION_MINUTE',
            'PARTY_AGE','VEHICLE_YEAR','PARTY_COUNT','DISTANCE']


collisions_clean_make[str_cols] = collisions_clean_make[str_cols].astype(float).astype('int64')

In [64]:
# change object data types to categorical 
to_cat_list = ['AT_FAULT', 'PARTY_SEX','PARTY_SOBRIETY',
               'PARTY_DRUG_PHYSICAL', 'DIR_OF_TRAVEL', 'PARTY_SAFETY_EQUIP_1',
               'PARTY_SAFETY_EQUIP_2', 'FINAN_RESPONS', 'SP_INFO_2', 'OAF_VIOL_CAT',
               'OAF_1', 'MOVE_PRE_ACC', 'VEHICLE_MAKE',
               'STWD_VEHICLE_TYPE', 'CHP_VEH_TYPE_TOWING', 'RACE', 'DAY_OF_WEEK',
               'CHP_SHIFT', 'POPULATION', 'BEAT_TYPE', 'CHP_BEAT_TYPE','DIRECTION', 
               'INTERSECTION', 'WEATHER_1', 'STATE_HWY_IND','PRIMARY_COLL_FACTOR', 'PCF_VIOL_CATEGORY',
               'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION', 'ROAD_SURFACE',
               'ROAD_COND_1', 'LIGHTING', 'CONTROL_DEVICE', 'ALCOHOL_INVOLVED',]

collisions_clean_make[to_cat_list] = collisions_clean_make[to_cat_list].apply(lambda x: x.astype('category'))

In [71]:
# make a new cleaned df
clean_collisions = collisions_clean_make.copy()

In [74]:
# export clean_collisions to a seperate notebook to save cpu memory
clean_collisions.to_csv('clean_collisions.csv')

# export to preserve data types
clean_collisions.to_parquet('clean_collisions.parquet')

This is the end of this Notebook.